In [1]:
%pip install snowflake-snowpark-python


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.38.0 requires cloudpickle<3,>=2.0.0, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.38.0 requires protobuf<4,>=3.12.2, but you have protobuf 5.29.5 which is incompatible.
apache-flink 1.16.0 requires cloudpickle==2.1.0, but you have cloudpickle 3.0.0 which is incompatible.
apache-flink 1.16.0 requires protobuf<3.18, but you have protobuf 5.29.5 which is incompatible.
apache-flink 1.16.0 requires python-dateutil==2.8.0, but you have python-dateutil 2.9.0.post0 which is incompatible.



   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------------------- -- 1.6/1.7 MB 9.3 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/949.1 kB ? eta -:--:--
   ---------------------------------------- 949.1/949.1 kB 5.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   ------------------ --------------------- 1.6/3.4 MB 9.3 MB/s eta 0:00:01
   ------------------------------------- -- 3.1/3.4 MB 7.7 MB/s eta 0:00:01
   ---------------------------------------- 3.4/3.4 MB 6.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/13.7 MB 7.7 MB/s eta 0:00:02
   ------- -------------------------------- 2.6/13.7 MB 6.0 MB/s eta 0:00:02
   ------------ --------------------------- 4.2/13.7 MB 6.6 MB/s eta 0:00:02
   --------------- ----------

In [2]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# from snowflake.snowpark import Session

# # Snowflake connection parameters
# connection_parameters = {
#     "account": "your_account",
#     "user": "your_user",
#     "password": "your_password",
#     "role": "your_role",
#     "warehouse": "COMPUTE_WH",
#     "database": "MOVIES_DB",
#     "schema": "PUBLIC"
# }

# # Establish Snowflake session
# session = Session.builder.configs(connection_parameters).create()


ModuleNotFoundError: No module named 'streamlit'

In [ ]:
# Load the raw_movie_bookings table
raw_movie_bookings = session.table("raw_movie_bookings")

# Show a preview of the data
raw_movie_bookings.show()

In [ ]:
from snowflake.snowpark.functions import col, lit, when

# Filter completed bookings
completed_bookings = raw_movie_bookings.filter(col("status") == "COMPLETED")

# Add a total_price column and high_value flag
transformed_df = completed_bookings.with_column(
    "total_price", col("ticket_count") * col("ticket_price")
).with_column(
    "high_value", when(col("total_price") > 50, lit(True)).otherwise(lit(False))
)

# Show transformed data
transformed_df.show()

In [ ]:
# Save transformed data to a new table
transformed_df.write.save_as_table("high_value_bookings", mode="overwrite")

In [ ]:
from snowflake.snowpark.functions import sum, avg

# Aggregate data
aggregated_df = raw_movie_bookings.group_by("movie_id").agg(
    sum(col("ticket_count")).alias("total_tickets"),
    avg(col("ticket_price")).alias("avg_ticket_price"),
    sum((col("ticket_count") * col("ticket_price"))).alias("total_revenue")
)

# Show aggregated data
aggregated_df.show()

In [ ]:
from snowflake.snowpark.types import IntegerType, StringType
from snowflake.snowpark.functions import udf

# Define a UDF to classify movies
@udf(return_type=StringType(), input_types=[IntegerType()])
def classify_revenue(total_revenue):
    if total_revenue > 100:
        return "Blockbuster"
    elif total_revenue > 50:
        return "Hit"
    else:
        return "Average"

# Apply the UDF
classified_df = aggregated_df.with_column(
    "movie_category", classify_revenue(col("total_revenue"))
)

# Show classified data
classified_df.show()